In [ ]:
#5個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route.xlsx"
filename2=("fire_route.xlsx")
N_number=5
T_number =70                         #時間長度
K = set([1,2])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([1])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



In [ ]:
df = pd.read_excel(filename1)
df_num = len(df.index)
for i in N:
    A_p.append((i,i))
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    fighterIndex = df.iloc[i]['k']
    time = df.iloc[i]['travel time']
    if (u,v) not in A_p:
        A_p.append((u,v))
    if mode == "random":
        tau[u,v,fighterIndex] = rand_fightertraveltime[i]
    else:
        tau[u,v,fighterIndex] = time   
       
    
df = pd.read_excel(filename2)
df_num = len(df.index)
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    time = df.iloc[i]['travel time']
    if (u,v) not in A_f:
        A_f.append((u,v))
    if mode == "random":
        lamb[u,v] = rand_firetraveltime[i]   #隨機產生資料用這行
    else:
        lamb[u,v] = time   #excel給定用這行
    
# print(tau)
# print(lamb)

A_f_NEIGHBOR={}                      #A_f_NEIGHBOR=與點i相鄰的點
    
for l in N-N_D:                          #定義A_f_NEIGHBOR
    connect = A_f.select('*',l)
    A_f_NEIGHBOR[l]=[]
    for temp in connect:
        A_f_NEIGHBOR[l].append(temp[0])
                      
A_f_NEIGHBOR_T={}                    #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點

for j in N-N_D-N_F:                         #定義A_f_NEIGHBOR_T
    for t in T:
        A_f_NEIGHBOR_T[j,t]=[]
        for i in A_f_NEIGHBOR[j]:
            if t-H[i]-lamb[i,j]>=0:
                A_f_NEIGHBOR_T[j,t].append(i)
model=gp.Model("FIREFIGHTER")


x={}
w={}
u={}
u_bar={}
v={}
v_bar={}
epsilon=1e-3

#定義x[i,j,k,t]
for k in K:
    for t in T:
        for i in range(len(A_p)):
                x[A_p[i][0],A_p[i][1],k,t]=model.addVar(vtype='B',name="x[%d,%d,%d,%d]" %(A_p[i][0],A_p[i][1],k,t))

#定義w[i,k,t]
for k in K:
    for t in T:
        for i in N:
            w[i,k,t]=model.addVar(vtype='B',name="w[%d,%d,%d]" %(i,k,t))

#定義u[i,t]
u = model.addVars(N, T, vtype="B",name="u")

#定義u_bar[i,k,t]
u_bar = model.addVars(N, K, T, vtype="B", name="u_bar")

#定義v[i,t]
v = model.addVars(N, T, vtype="B", name="v")
        
#定義v_bar[i,t]
v_bar = model.addVars(N, T, vtype="B", name="v_bar")        
        
model.update()

for k in K:                        #原點flow blance
    model.addConstr(gp.quicksum(x[i,j,k,0] for i,j in A_p) <= 1)
    
#depart from depot
for O in N_D:
    connect = A_p.select(O,'*')
    for k in K:
        model.addConstr(gp.quicksum(x[i,j,k,0] for i, j in connect) == 1)
#depot can't be processed
model.addConstrs((u_bar[i,k,t]==0 for i in N_D for k in K for t in range(T_number+1)))

#flow balance
for k in K:
    for t in range(1,T_number):
        for j in N: 
            in_connect = A_p.select('*',j)
            out_connect = A_p.select(j,'*')
            temp=0 #in-degree
            temp += w[j,k,t-1] # t-1在j idle
            if j in N_D: #j in depot set會有u_bar，只是都為0
                temp+=u_bar[j,k,t]
            else: #j not in depot set若現在的t>process time，則會有u_bar
                if process[k][j] <= t:
                    temp += u_bar[j,k,t-process[k][j]]
            for m,n in in_connect:
                if m != n and tau[m,n,k] <= t: #若現在的t>travel time，則會有x
                    temp += x[m,n,k,t-tau[m,n,k]]
            model.addConstr(temp == gp.quicksum(x[n,w,k,t] for n,w in out_connect),name="flow") #in-degree = out-degree
model.addConstrs((u_bar[i,k,t] + w[i,k,t] == x[i,i,k,t] for i in N-N_D for k in K for t in range(T_number)))
model.addConstrs((w[s,k,t] == x[s,s,k,t]) for s in N_D for k in K for t in range(T_number))

#######################################################################################################################
model.addConstrs(u[i, t]+u_bar.sum(i, '*', t) <= 1 for i in N for t in T) #constrain 6
model.addConstrs(v[i, t] + u[i, t] == v[i, t+1] for i in N for t in T[0:-1]) #constrain 7
model.addConstrs(v_bar[i, t] + u_bar.sum(i, '*', t) == v_bar[i, t+1] for i in N for t in T[0:-1]) #constrain 9
model.addConstrs(u_bar.sum(s, '*', '*') == 0 for s in N_D) #constrain 11
model.addConstrs(u.sum(s, '*') == 0 for s in N_D) #constrain 12
        
#######################################################################################################################
    
for j in N-N_D-N_F:                            #火焰的延燒
    for t in range(T_number):
        if len(A_f_NEIGHBOR_T[j,t])==0:
            model.addConstr(u[j,t] == 0,name='test')
        else:
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) / M <= u[j,t] + v[j,t] + v_bar[j,t+1])
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) >= u[j,t])
            
for i in N_F:                           #起火點起火
    model.addConstr(u[i,0]==1)
    
for i in N:                            #所有節點不會在t-1~t已經被燒
    model.addConstr(v[i,0]==0)
    
for i in N:                            #所有節點不會在t-1~t已經被澆水
    model.addConstr(v_bar[i,0]==0)
            
for k in K:                             #消防員不能去已經被燃燒的節點
    for t in T:
        for l in range(len(A_p)):
            if A_p[l][1] not in N_D:
                if A_p[l][0] ==  A_p[l][1]:
                    if t+2 <= T_number:
                        model.addConstr(M*(1-v[A_p[l][1],t+1]) >= x[A_p[l][0], A_p[l][1],k,t])
                    else:
                        model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0], A_p[l][1],k,t])
                elif t + tau[A_p[l][0],A_p[l][1],k] + 1 <= T_number:
                    model.addConstr(M*(1-v[A_p[l][1],t + tau[A_p[l][0],A_p[l][1],k]]) >= x[A_p[l][0],A_p[l][1],k,t])
                else:
                    model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0],A_p[l][1],k,t])
                    
                    
model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D) + 
                   gp.quicksum(epsilon*x[i,j,k,t] for (i,j,k,t) in x if i!=j),GRB.MINIMIZE)


model.optimize()
model.write('test.lp')
print("optimal value : ",model.ObjVal)
                    

In [ ]:
EPS=1.e-2
print("x:")
step=1
for k in K:
    print()
    print("消防員%d的路徑" % k)
    temp = [elem for elem in x if elem[2]==k]
    for (i,j,k,t) in temp:
        if x[i,j,k,t].X >EPS:
            #print("x[%d,%d,%d,%d]" %(i,j,k,t) , x[i,j,k,t].X)
            if i != j:
                print("在時刻 %d 從node%d 移動到 node%d" %(t,i,j)," ,travel time:",tau[i,j,k])
            else:            
                if u_bar[i,k,t].X == 1:
                    print("在時刻 %d 對node%d進行保護"%(t,i)," ,processing time:",process[k][i])
                else:
                    print("在時刻 %d 在node%d idle" %(t,i))

print("w:")
for (i,k,t) in w:
    if w[i,k,t].X>EPS:
        print("w[%d,%d,%d]" %(i,k,t) , w[i,k,t].X)

print("u:")
for (i,t) in u:
    if u[i,t].X>EPS:
        print("u[%d,%d]" %(i,t) , u[i,t].X)

print("u_bar:")
for (i,k,t) in u_bar:
    if u_bar[i,k,t].X>EPS:
        print("u_bar[%d,%d,%d]" %(i,k,t) , u_bar[i,k,t].X)

print("v:")
for (i,t) in v:
    if v[i,t].X>EPS:
        print("v[%d,%d]" %(i,t) , v[i,t].X)

print("v_bar:")
for (i,t) in v_bar:
    if v_bar[i,t].X>EPS:
        print("v_bar[%d,%d]" %(i,t) , v_bar[i,t].X) 

In [ ]:
#6個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_2.xlsx"
filename2=("fire_route_2.xlsx")
N_number=6
T_number =70                         #時間長度
K = set([1,2,3])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([6])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4,3:5}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



In [ ]:
#7個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_3.xlsx"
filename2=("fire_route_3.xlsx")
N_number=7
T_number =70                         #時間長度
K = set([1,2])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([3,5,7])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:5}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3,7:3}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50,7:100}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1,7:3}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



In [ ]:
#10個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_4.xlsx"
filename2=("fire_route_4.xlsx")
N_number=10
T_number =70                         #時間長度
K = set([1,2])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([6,9])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:5}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3,7:3,8:3,9:2,10:7}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50,7:100,8:10,9:20,10:15}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1,7:3,8:2,9:4,10:6}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



In [ ]:
#15個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_5.xlsx"
filename2=("fire_route_5.xlsx")
N_number=15
T_number =70                         #時間長度
K = set([1,2])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([12,5,1])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3,7:3,8:3,9:2,10:7,11:3,12:3,13:3,14:6,15:6}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50,7:100,8:10,9:20,10:15,11:10,12:10,13:10,14:5,15:35}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1,7:3,8:2,9:4,10:6,11:7,12:5,13:2,14:2,15:5}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



In [ ]:
#20個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_6.xlsx"
filename2=("fire_route_6.xlsx")
N_number=20
T_number =70                         #時間長度
K = set([1,2,3])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([10,18])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4,3:2}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3,7:3,8:3,9:2,10:7,11:3,12:3,13:3,14:6,15:6,16:3,17:3,18:3,19:6,20:6}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50,7:100,8:10,9:20,10:15,11:10,12:10,13:10,14:5,15:35,16:10,17:10,18:10,19:5,20:35}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1,7:3,8:2,9:4,10:6,11:7,12:5,13:2,14:2,15:5,16:7,17:5,18:2,19:2,20:5}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



In [1]:
#30個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="G30_firefighter_route.xlsx"
filename2=("G30_fire_route.xlsx")
filename3=("G30_nodeInformation.xlsx")
N_number=30
T_number = 150                         #時間長度
K = set([1])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([22])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3}                                  #各個消防車單位時間處理的燃料量

"""
if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    #Q={1:6,2:2,3:4,5:6,6:3,7:3,8:3,9:2,10:7,11:3,12:3,13:3,14:6,15:6,16:3,17:3,18:3,19:6,20:6}                  #q=單位時間火在i點燒的燃料量
    #b={1:18,2:8,3:12,5:18,6:50,7:100,8:10,9:20,10:15,11:10,12:10,13:10,14:5,15:35,16:10,17:10,18:10,19:5,20:35}               #b=節點i的價值
    #H={1:2,2:3,3:3,5:2,6:1,7:3,8:2,9:4,10:6,11:7,12:5,13:2,14:2,15:5,16:7,17:5,18:2,19:2,20:5}                  #h=火燒i點需要的時間
    Q={}
    b={}
    H={}
    
    for i in range(1,N_number+1):
        if i not in N_D:
            Q[i] = random.randint(5,8)
            #Q[i] = 5
            b[i] = random.choice([5,10,15])
            #b[i]=30
            H[i] = random.randint(5,8)
            #H[i] = 5
"""

Q={}
b={}
H={}

df = pd.read_excel(filename3)
df_num = len(df.index)

for i in range(0,df_num):
    value = df.iloc[i]['value']
    h = df.iloc[i]['burning time']
    q = df.iloc[i]['quantity']
    Q[i + 1] = q
    b[i + 1] = value
    H[i + 1] = h

T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)
print(tau)



{1: {1: 12, 2: 17, 3: 14, 4: 10, 5: 12, 6: 19, 7: 10, 8: 16, 9: 12, 10: 14, 11: 10, 12: 14, 13: 12, 14: 19, 15: 22, 16: 12, 17: 9, 18: 14, 19: 14, 20: 19, 21: 19, 22: 17, 23: 10, 24: 16, 25: 14, 26: 14, 27: 22, 28: 19, 29: 14, 30: 14}}
Q: {1: 5, 2: 7, 3: 7, 4: 5, 5: 6, 6: 8, 7: 5, 8: 6, 9: 7, 10: 8, 11: 5, 12: 7, 13: 5, 14: 8, 15: 8, 16: 6, 17: 5, 18: 8, 19: 5, 20: 8, 21: 7, 22: 7, 23: 5, 24: 8, 25: 6, 26: 7, 27: 8, 28: 7, 29: 7, 30: 5}
H: {1: 7, 2: 7, 3: 6, 4: 6, 5: 6, 6: 7, 7: 6, 8: 8, 9: 5, 10: 5, 11: 6, 12: 6, 13: 7, 14: 7, 15: 8, 16: 6, 17: 5, 18: 5, 19: 8, 20: 7, 21: 8, 22: 7, 23: 6, 24: 6, 25: 7, 26: 6, 27: 8, 28: 8, 29: 6, 30: 8}
b: {1: 10, 2: 15, 3: 10, 4: 5, 5: 15, 6: 10, 7: 15, 8: 15, 9: 15, 10: 5, 11: 5, 12: 10, 13: 5, 14: 15, 15: 5, 16: 10, 17: 15, 18: 10, 19: 10, 20: 5, 21: 5, 22: 15, 23: 5, 24: 5, 25: 5, 26: 5, 27: 5, 28: 5, 29: 10, 30: 5}
{}


In [2]:
df = pd.read_excel(filename1)
df_num = len(df.index)
for i in N:
    A_p.append((i,i))
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    fighterIndex = df.iloc[i]['k']
    time = df.iloc[i]['travel time']
    if (u,v) not in A_p:
        A_p.append((u,v))
    if mode == "random":
        tau[u,v,fighterIndex] = rand_fightertraveltime[i]
    else:
        tau[u,v,fighterIndex] = time   
       
    
df = pd.read_excel(filename2)
df_num = len(df.index)
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    time = df.iloc[i]['travel time']
    if (u,v) not in A_f:
        A_f.append((u,v))
    if mode == "random":
        lamb[u,v] = rand_firetraveltime[i]   #隨機產生資料用這行
    else:
        lamb[u,v] = time   #excel給定用這行
    
# print(tau)
# print(lamb)

A_f_NEIGHBOR={}                      #A_f_NEIGHBOR=與點i相鄰的點
    
for l in N-N_D:                          #定義A_f_NEIGHBOR
    connect = A_f.select('*',l)
    A_f_NEIGHBOR[l]=[]
    for temp in connect:
        A_f_NEIGHBOR[l].append(temp[0])
                      
A_f_NEIGHBOR_T={}                    #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點

for j in N-N_D-N_F:                         #定義A_f_NEIGHBOR_T
    for t in T:
        A_f_NEIGHBOR_T[j,t]=[]
        for i in A_f_NEIGHBOR[j]:
            if t-H[i]-lamb[i,j]>=0:
                A_f_NEIGHBOR_T[j,t].append(i)
model=gp.Model("FIREFIGHTER")


x={}
w={}
u={}
u_bar={}
v={}
v_bar={}
epsilon=1e-4

#定義x[i,j,k,t]
for k in K:
    for t in T:
        for i in range(len(A_p)):
                x[A_p[i][0],A_p[i][1],k,t]=model.addVar(vtype='B',name="x[%d,%d,%d,%d]" %(A_p[i][0],A_p[i][1],k,t))

#定義w[i,k,t]
for k in K:
    for t in T:
        for i in N:
            w[i,k,t]=model.addVar(vtype='B',name="w[%d,%d,%d]" %(i,k,t))

#定義u[i,t]
u = model.addVars(N, T, vtype="B",name="u")

#定義u_bar[i,k,t]
u_bar = model.addVars(N, K, T, vtype="B", name="u_bar")

#定義v[i,t]
v = model.addVars(N, T, vtype="B", name="v")
        
#定義v_bar[i,t]
v_bar = model.addVars(N, T, vtype="B", name="v_bar")        
        
model.update()

for k in K:                        #原點flow blance
    model.addConstr(gp.quicksum(x[i,j,k,0] for i,j in A_p) <= 1)
    
#depart from depot
for O in N_D:
    connect = A_p.select(O,'*')
    for k in K:
        model.addConstr(gp.quicksum(x[i,j,k,0] for i, j in connect) == 1)
#depot can't be processed
model.addConstrs((u_bar[i,k,t]==0 for i in N_D for k in K for t in range(T_number+1)))

#flow balance
for k in K:
    for t in range(1,T_number):
        for j in N: 
            in_connect = A_p.select('*',j)
            out_connect = A_p.select(j,'*')
            temp=0 #in-degree
            temp += w[j,k,t-1] # t-1在j idle
            if j in N_D: #j in depot set會有u_bar，只是都為0
                temp+=u_bar[j,k,t]
            else: #j not in depot set若現在的t>process time，則會有u_bar
                if process[k][j] <= t:
                    temp += u_bar[j,k,t-process[k][j]]
            for m,n in in_connect:
                if m != n and tau[m,n,k] <= t: #若現在的t>travel time，則會有x
                    temp += x[m,n,k,t-tau[m,n,k]]
            model.addConstr(temp == gp.quicksum(x[n,w,k,t] for n,w in out_connect),name="flow") #in-degree = out-degree
model.addConstrs((u_bar[i,k,t] + w[i,k,t] == x[i,i,k,t] for i in N-N_D for k in K for t in range(T_number)))
model.addConstrs((w[s,k,t] == x[s,s,k,t]) for s in N_D for k in K for t in range(T_number))

#######################################################################################################################
model.addConstrs(u[i, t]+u_bar.sum(i, '*', t) <= 1 for i in N for t in T) #constrain 6
model.addConstrs(v[i, t] + u[i, t] == v[i, t+1] for i in N for t in T[0:-1]) #constrain 7
model.addConstrs(v_bar[i, t] + u_bar.sum(i, '*', t) == v_bar[i, t+1] for i in N for t in T[0:-1]) #constrain 9
model.addConstrs(u_bar.sum(s, '*', '*') == 0 for s in N_D) #constrain 11
model.addConstrs(u.sum(s, '*') == 0 for s in N_D) #constrain 12
        
#######################################################################################################################
    
for j in N-N_D-N_F:                            #火焰的延燒
    for t in range(T_number):
        if len(A_f_NEIGHBOR_T[j,t])==0:
            model.addConstr(u[j,t] == 0,name='test')
        else:
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) / M <= u[j,t] + v[j,t] + v_bar[j,t+1])
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) >= u[j,t])
            
for i in N_F:                           #起火點起火
    model.addConstr(u[i,0]==1)
    
for i in N:                            #所有節點不會在t-1~t已經被燒
    model.addConstr(v[i,0]==0)
    
for i in N:                            #所有節點不會在t-1~t已經被澆水
    model.addConstr(v_bar[i,0]==0)
            
for k in K:                             #消防員不能去已經被燃燒的節點
    for t in T:
        for l in range(len(A_p)):
            if A_p[l][1] not in N_D:
                if A_p[l][0] ==  A_p[l][1]:
                    if t+2 <= T_number:
                        model.addConstr(M*(1-v[A_p[l][1],t+1]) >= x[A_p[l][0], A_p[l][1],k,t])
                    else:
                        model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0], A_p[l][1],k,t])
                elif t + tau[A_p[l][0],A_p[l][1],k] + 1 <= T_number:
                    model.addConstr(M*(1-v[A_p[l][1],t + tau[A_p[l][0],A_p[l][1],k]]) >= x[A_p[l][0],A_p[l][1],k,t])
                else:
                    model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0],A_p[l][1],k,t])
                    
                    
model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D) + 
                   gp.quicksum(epsilon*x[i,j,k,t] for (i,j,k,t) in x if i!=j),GRB.MINIMIZE)


model.optimize()
model.write('test.lp')
print("optimal value : ",model.ObjVal)
                    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-26
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 55205 rows, 48018 columns and 199394 nonzeros
Model fingerprint: 0x01f7ac65
Variable types: 0 continuous, 48018 integer (48018 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 40149 rows and 29285 columns
Presolve time: 2.58s
Presolved: 15056 rows, 18733 columns, 60339 nonzeros
Variable types: 0 continuous, 18733 integer (18733 binary)
Found heuristic solution: objective 255.0008000

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    9321    1.6866935e+02   1.092721e+03   0.000000e+00      5s
   12657    1.8127826e+02   0.000000e+00   0.000000e+00      8s

Root relaxation: ob

In [3]:
EPS=1.e-2
print("x:")
step=1
for k in K:
    print()
    print("消防員%d的路徑" % k)
    temp = [elem for elem in x if elem[2]==k]
    for (i,j,k,t) in temp:
        if x[i,j,k,t].X >EPS:
            #print("x[%d,%d,%d,%d]" %(i,j,k,t) , x[i,j,k,t].X)
            if i != j:
                print("在時刻 %d 從node%d 移動到 node%d" %(t,i,j)," ,travel time:",tau[i,j,k])
            else:            
                if u_bar[i,k,t].X == 1:
                    print("在時刻 %d 對node%d進行保護"%(t,i)," ,processing time:",process[k][i])
                else:
                    print("在時刻 %d 在node%d idle" %(t,i))

print("w:")
for (i,k,t) in w:
    if w[i,k,t].X>EPS:
        print("w[%d,%d,%d]" %(i,k,t) , w[i,k,t].X)

print("u:")
for (i,t) in u:
    if u[i,t].X>EPS:
        print("u[%d,%d]" %(i,t) , u[i,t].X)

print("u_bar:")
for (i,k,t) in u_bar:
    if u_bar[i,k,t].X>EPS:
        print("u_bar[%d,%d,%d]" %(i,k,t) , u_bar[i,k,t].X)

print("v:")
for (i,t) in v:
    if v[i,t].X>EPS:
        print("v[%d,%d]" %(i,t) , v[i,t].X)

print("v_bar:")
for (i,t) in v_bar:
    if v_bar[i,t].X>EPS:
        print("v_bar[%d,%d]" %(i,t) , v_bar[i,t].X) 

x:

消防員1的路徑
在時刻 0 從node4 移動到 node23  ,travel time: 6.0
在時刻 6 從node23 移動到 node29  ,travel time: 26.0
在時刻 32 從node29 移動到 node17  ,travel time: 9.0
在時刻 41 對node17進行保護  ,processing time: 9
在時刻 50 從node17 移動到 node29  ,travel time: 9.0
在時刻 59 在node29 idle
在時刻 60 對node29進行保護  ,processing time: 14
在時刻 74 從node29 移動到 node24  ,travel time: 10.0
在時刻 84 對node24進行保護  ,processing time: 16
在時刻 100 在node24 idle
在時刻 101 從node24 移動到 node18  ,travel time: 10.0
在時刻 111 在node18 idle
在時刻 112 對node18進行保護  ,processing time: 14
在時刻 126 在node18 idle
在時刻 127 在node18 idle
在時刻 128 在node18 idle
在時刻 129 在node18 idle
在時刻 130 在node18 idle
在時刻 131 在node18 idle
在時刻 132 在node18 idle
在時刻 133 在node18 idle
在時刻 134 在node18 idle
在時刻 135 在node18 idle
在時刻 136 在node18 idle
在時刻 137 在node18 idle
在時刻 138 在node18 idle
在時刻 139 在node18 idle
在時刻 140 在node18 idle
在時刻 141 在node18 idle
在時刻 142 在node18 idle
在時刻 143 在node18 idle
在時刻 144 在node18 idle
在時刻 145 在node18 idle
在時刻 146 在node18 idle
在時刻 147 在node18 idle
在時刻 148 在node18 idle
在時刻 149 

v[8,107] 1.0
v[8,108] 1.0
v[8,109] 1.0
v[8,110] 1.0
v[8,111] 1.0
v[8,112] 1.0
v[8,113] 1.0
v[8,114] 1.0
v[8,115] 1.0
v[8,116] 1.0
v[8,117] 1.0
v[8,118] 1.0
v[8,119] 1.0
v[8,120] 1.0
v[8,121] 1.0
v[8,122] 1.0
v[8,123] 1.0
v[8,124] 1.0
v[8,125] 1.0
v[8,126] 1.0
v[8,127] 1.0
v[8,128] 1.0
v[8,129] 1.0
v[8,130] 1.0
v[8,131] 1.0
v[8,132] 1.0
v[8,133] 1.0
v[8,134] 1.0
v[8,135] 1.0
v[8,136] 1.0
v[8,137] 1.0
v[8,138] 1.0
v[8,139] 1.0
v[8,140] 1.0
v[8,141] 1.0
v[8,142] 1.0
v[8,143] 1.0
v[8,144] 1.0
v[8,145] 1.0
v[8,146] 1.0
v[8,147] 1.0
v[8,148] 1.0
v[8,149] 1.0
v[8,150] 1.0
v[10,76] 1.0
v[10,77] 1.0
v[10,78] 1.0
v[10,79] 1.0
v[10,80] 1.0
v[10,81] 1.0
v[10,82] 1.0
v[10,83] 1.0
v[10,84] 1.0
v[10,85] 1.0
v[10,86] 1.0
v[10,87] 1.0
v[10,88] 1.0
v[10,89] 1.0
v[10,90] 1.0
v[10,91] 1.0
v[10,92] 1.0
v[10,93] 1.0
v[10,94] 1.0
v[10,95] 1.0
v[10,96] 1.0
v[10,97] 1.0
v[10,98] 1.0
v[10,99] 1.0
v[10,100] 1.0
v[10,101] 1.0
v[10,102] 1.0
v[10,103] 1.0
v[10,104] 1.0
v[10,105] 1.0
v[10,106] 1.0
v[10,107] 1.0
v[10

v[20,86] 1.0
v[20,87] 1.0
v[20,88] 1.0
v[20,89] 1.0
v[20,90] 1.0
v[20,91] 1.0
v[20,92] 1.0
v[20,93] 1.0
v[20,94] 1.0
v[20,95] 1.0
v[20,96] 1.0
v[20,97] 1.0
v[20,98] 1.0
v[20,99] 1.0
v[20,100] 1.0
v[20,101] 1.0
v[20,102] 1.0
v[20,103] 1.0
v[20,104] 1.0
v[20,105] 1.0
v[20,106] 1.0
v[20,107] 1.0
v[20,108] 1.0
v[20,109] 1.0
v[20,110] 1.0
v[20,111] 1.0
v[20,112] 1.0
v[20,113] 1.0
v[20,114] 1.0
v[20,115] 1.0
v[20,116] 1.0
v[20,117] 1.0
v[20,118] 1.0
v[20,119] 1.0
v[20,120] 1.0
v[20,121] 1.0
v[20,122] 1.0
v[20,123] 1.0
v[20,124] 1.0
v[20,125] 1.0
v[20,126] 1.0
v[20,127] 1.0
v[20,128] 1.0
v[20,129] 1.0
v[20,130] 1.0
v[20,131] 1.0
v[20,132] 1.0
v[20,133] 1.0
v[20,134] 1.0
v[20,135] 1.0
v[20,136] 1.0
v[20,137] 1.0
v[20,138] 1.0
v[20,139] 1.0
v[20,140] 1.0
v[20,141] 1.0
v[20,142] 1.0
v[20,143] 1.0
v[20,144] 1.0
v[20,145] 1.0
v[20,146] 1.0
v[20,147] 1.0
v[20,148] 1.0
v[20,149] 1.0
v[20,150] 1.0
v[21,88] 1.0
v[21,89] 1.0
v[21,90] 1.0
v[21,91] 1.0
v[21,92] 1.0
v[21,93] 1.0
v[21,94] 1.0
v[21,95] 1.0


v_bar[17,42] 1.0
v_bar[17,43] 1.0
v_bar[17,44] 1.0
v_bar[17,45] 1.0
v_bar[17,46] 1.0
v_bar[17,47] 1.0
v_bar[17,48] 1.0
v_bar[17,49] 1.0
v_bar[17,50] 1.0
v_bar[17,51] 1.0
v_bar[17,52] 1.0
v_bar[17,53] 1.0
v_bar[17,54] 1.0
v_bar[17,55] 1.0
v_bar[17,56] 1.0
v_bar[17,57] 1.0
v_bar[17,58] 1.0
v_bar[17,59] 1.0
v_bar[17,60] 1.0
v_bar[17,61] 1.0
v_bar[17,62] 1.0
v_bar[17,63] 1.0
v_bar[17,64] 1.0
v_bar[17,65] 1.0
v_bar[17,66] 1.0
v_bar[17,67] 1.0
v_bar[17,68] 1.0
v_bar[17,69] 1.0
v_bar[17,70] 1.0
v_bar[17,71] 1.0
v_bar[17,72] 1.0
v_bar[17,73] 1.0
v_bar[17,74] 1.0
v_bar[17,75] 1.0
v_bar[17,76] 1.0
v_bar[17,77] 1.0
v_bar[17,78] 1.0
v_bar[17,79] 1.0
v_bar[17,80] 1.0
v_bar[17,81] 1.0
v_bar[17,82] 1.0
v_bar[17,83] 1.0
v_bar[17,84] 1.0
v_bar[17,85] 1.0
v_bar[17,86] 1.0
v_bar[17,87] 1.0
v_bar[17,88] 1.0
v_bar[17,89] 1.0
v_bar[17,90] 1.0
v_bar[17,91] 1.0
v_bar[17,92] 1.0
v_bar[17,93] 1.0
v_bar[17,94] 1.0
v_bar[17,95] 1.0
v_bar[17,96] 1.0
v_bar[17,97] 1.0
v_bar[17,98] 1.0
v_bar[17,99] 1.0
v_bar[17,100] 